<a href="https://colab.research.google.com/github/jsleroux/ReinforcementLearningProjects/blob/master/FrozenLake_Markov_Decision_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install torch==1.0.0 > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor

gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from collections import deque
%matplotlib inline

import math
import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
# Start virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)


xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

### Call the environment

In [0]:
env = (gym.make("FrozenLake-v0"))

In [0]:
discount = 0.8
epsilon = 1e-3


values = np.zeros([e.env.observation_space.n])
values[15] = 1

termS = [5,7,11,12,15]

e = env
model = e.env.P

In [0]:
R = np.zeros([env.env.action_space.n, env.env.observation_space.n])
R[:, 15] = 1


T = np.zeros([env.action_space.n, env.observation_space.n, env.observation_space.n])
for state in model:
    for action in model[state]:
        for nextstate in model[state][action]:
            T[action, state, nextstate[1]] = nextstate[0]

In [0]:
def getReward(rewards, state):
    return rewards[state]

In [0]:
def computeQValueFromValues(s, a):
    qval = T[a, s, :].dot(R[a, :] + (discount*values_[:]))
         
    return qval

In [0]:
i = 0
while True:
    i+=1
    delta = 0
    values_ = values.copy()
    for state in range(e.observation_space.n):
        allactionrewards = np.zeros([e.env.action_space.n])
        for action in range(e.action_space.n):
            allactionrewards[action] = computeQValueFromValues(state, action)
        if state not in termS:
            values[state] = allactionrewards.max()
        
        if np.abs(values[state]-values_[state]) > delta:
            delta = np.abs(values[state]-values_[state])
            
    if delta < epsilon * (1 - discount) / discount:
        break

print('Learned in {} iterations'.format(i))

Learned in 22 iterations


In [0]:
print(values[0:4])
print(values[4:8])
print(values[8:12])
print(values[12:16])

[0.02704473 0.02739048 0.04882192 0.0176735 ]
[0.04760031 0.         0.10728956 0.        ]
[0.10448939 0.24032175 0.3537701  0.        ]
[0.         0.44338616 0.97934178 1.        ]


In [0]:
# importing dependency libraries
from __future__ import print_function
import gym
import numpy as np
import time

#Load the environment
env = gym.make('FrozenLake-v0')

s = env.reset()
print(s)
print()

env.render()
print()

print(env.action_space) #number of actions
print(env.observation_space) #number of states
print()

print("Number of actions : ",env.action_space.n)
print("Number of states : ",env.observation_space.n)
print()

# Value Iteration Implementation

#Initializing Utilities of all states with zeros
U = np.zeros([env.observation_space.n])

#since terminal states have utility values equal to their reward
U[15] = 1 #goal state
U[[5,7,11,12]] = 0 #hole states
termS = [5,7,11,12,15] #terminal states
#set hyperparameters
y = 0.8 #discount factor lambda

eps = 1e-3 #threshold if the learning difference i.e. prev_u - U goes below this value break the learning

i=0
while(True):
    i+=1
    prev_u = np.copy(U)
    for s in range(env.observation_space.n):
        q_sa = [sum([p*(r + y*prev_u[s_]) for p, s_, r, _ in env.env.P[s][a]]) for a in range(env.action_space.n)]
        if s not in termS: 
            U[s] = max(q_sa)
            
    if (np.sum(np.fabs(prev_u - U)) <= eps):
        print ('Value-iteration converged at iteration# %d.' %(i+1))
        break

print("After learning completion printing the utilities for each states below from state ids 0-15")
print()
print(U[:4])
print(U[4:8])
print(U[8:12])
print(U[12:16])

0


SFFF
FHFH
FFFH
HFFG

Discrete(4)
Discrete(16)

Number of actions :  4
Number of states :  16

Value-iteration converged at iteration# 25.
After learning completion printing the utilities for each states below from state ids 0-15

[0.02739138 0.02774255 0.04918606 0.02798151]
[0.0479302  0.         0.10746661 0.        ]
[0.10477919 0.24054144 0.35393967 0.        ]
[0.         0.44355711 0.9794359  1.        ]
